In [ ]:
import os
import requests
from bs4 import BeautifulSoup

# === CONFIGURATION ===
BASE_URL = "https://sleepdata.org/datasets/nchsdb/files/sleep_data"
SAVE_DIR = "/tf/rahman/NCH_Sleep_Data_Bank/data/"
os.makedirs(SAVE_DIR, exist_ok=True)

# Paste your current session cookie here
SESSION_COOKIE = "_www_sleepdata_org_session=hyEH%2BitP3XzDC9biMVf67HqNeooCr8ROmAWVTvFPIE33vsxezroKOIrkeVMd%2BrWKf%2BDDG%2BLiD1aR0asaO%2FEzLpx22Kdu8T77Ojy%2FzuY7XhAYeHjFxF1zJVvzIdcht2BaGORNX4XlaiYwajK0LZrzh4DDFJdglTJ6bpbF4mpd%2Barjb%2FirlRgr8V65pRJvL9aSRCeX5OoXdnOUQMGJHuvE5rWCYt%2FA3IeUf982xiLb%2B3%2FvOrLqqzBzohT4Sjq%2B8cuFGMH6i1qWnoaPCOx0XZDB4Uw7m8udjAv5GwIgjNf%2FfU1XJDXDmyLZP%2BKLjLPP9gznICLYJldx5cNK6xnIBHqfSKdzJL5y%2FUh4lYUPL%2FW5LEAZmIJiG1DHEMeW69Ni5Al52hkhMJ9b2N1dImibNAx0HB2GVROTeNq1Zqq9jI6U%2F1bgSQR6444cwAB7E3i914AgJ6C92cMJi5OXz6Btw717uGRt8%2F%2BhgpZTgHRTkV05x1hFLIIuq%2B%2FT7rF7VT2j%2Bc3JtAjs7KhzdyjiC6vzbnBaBKI92DvxSw8LyBt1L%2BlAQytjCVnF11a2zi9rRW3lwPhtpQvgDbDy9ppErEq%2BvA1xrC3oy3qATiR5u5h2VOiYJcZw4lTp8cnQ3xmDACW9WSIMfm%2BB1OJA5PCuNd2jW%2FCD5ypZrsyIN%2FRX7zGn05hr82zCOO6H9G59zU28GLoWnfgooGhjLwnB%2FwMgdMzTrTcWYm3BaS9rWN%2F7%2FjXOv2VqQgStnrWK8YCx%2FPoapJWV1v%2F3a95lxfeb--VpWBlndzTmgsRb9J--ZiqyJkT%2Bha5DyLgr01W99A%3D%3D"

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://sleepdata.org",
    "Cookie": "_www_sleepdata_org_session=hyEH%2BitP3XzDC9biMVf67HqNeooCr8ROmAWVTvFPIE33vsxezroKOIrkeVMd%2BrWKf%2BDDG%2BLiD1aR0asaO%2FEzLpx22Kdu8T77Ojy%2FzuY7XhAYeHjFxF1zJVvzIdcht2BaGORNX4XlaiYwajK0LZrzh4DDFJdglTJ6bpbF4mpd%2Barjb%2FirlRgr8V65pRJvL9aSRCeX5OoXdnOUQMGJHuvE5rWCYt%2FA3IeUf982xiLb%2B3%2FvOrLqqzBzohT4Sjq%2B8cuFGMH6i1qWnoaPCOx0XZDB4Uw7m8udjAv5GwIgjNf%2FfU1XJDXDmyLZP%2BKLjLPP9gznICLYJldx5cNK6xnIBHqfSKdzJL5y%2FUh4lYUPL%2FW5LEAZmIJiG1DHEMeW69Ni5Al52hkhMJ9b2N1dImibNAx0HB2GVROTeNq1Zqq9jI6U%2F1bgSQR6444cwAB7E3i914AgJ6C92cMJi5OXz6Btw717uGRt8%2F%2BhgpZTgHRTkV05x1hFLIIuq%2B%2FT7rF7VT2j%2Bc3JtAjs7KhzdyjiC6vzbnBaBKI92DvxSw8LyBt1L%2BlAQytjCVnF11a2zi9rRW3lwPhtpQvgDbDy9ppErEq%2BvA1xrC3oy3qATiR5u5h2VOiYJcZw4lTp8cnQ3xmDACW9WSIMfm%2BB1OJA5PCuNd2jW%2FCD5ypZrsyIN%2FRX7zGn05hr82zCOO6H9G59zU28GLoWnfgooGhjLwnB%2FwMgdMzTrTcWYm3BaS9rWN%2F7%2FjXOv2VqQgStnrWK8YCx%2FPoapJWV1v%2F3a95lxfeb--VpWBlndzTmgsRb9J--ZiqyJkT%2Bha5DyLgr01W99A%3D%3D",
    "Accept": "*/*",
    "Connection": "keep-alive"
}


def extract_files_from_page(page_number):
    url = f"{BASE_URL}?page={page_number}"
    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:
        print(f"Failed to access page {page_number} (status: {response.status_code})")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    files = []

    for link in soup.find_all("a", href=True):
        href = link["href"]
        if href.endswith(".edf") or href.endswith(".annot"):
            file_name = href.split("/")[-1]
            files.append(file_name)

    print(f"Page {page_number}: Found {len(files)} files")
    return files


def download_file(file_name):
    url = f"{BASE_URL}/{file_name}"
    local_path = os.path.join(SAVE_DIR, file_name)

    if os.path.exists(local_path) and os.path.getsize(local_path) > 0:
        print(f"{file_name} already exists and is non-empty, skipping.")
        return

    print(f"Downloading: {file_name}")
    try:
        with requests.get(url, headers=HEADERS, stream=True, timeout=300) as res:
            if res.status_code == 200:
                with open(local_path, "wb") as f:
                    for chunk in res.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)

                downloaded_size = os.path.getsize(local_path)
                print(f"{file_name} downloaded successfully. Size: {downloaded_size / (1024*1024):.2f} MB")
            else:
                print(f"Failed to download {file_name} (status: {res.status_code})")
    except Exception as e:
        print(f"Error downloading {file_name}: {e}")


def main():
    all_files = []

    for page_num in range(0, 3):  # You can change this range based on number of pages
        page_files = extract_files_from_page(page_num)
        all_files.extend(page_files)

    print(f"Total files to download: {len(all_files)}")

    for file_name in set(all_files):  # remove duplicates
        download_file(file_name)


if __name__ == "__main__":
    main()
